# Fiddler Ranking Model Quick Start Guide

Fiddler enables your teams to monitor and evaluate model rankings, providing insights into model performance and detecting issues like data drift before they impact your applications.

### Example: Expedia Search Ranking
The following dataset contains Expedia shopping and purchase data as well as information on price competitiveness. The data are organized around a set of “search result impressions”, or the ordered list of hotels that the user sees after they search for a hotel on the Expedia website. In addition to impressions from the existing algorithm, the data contain impressions where the hotels were randomly sorted, to avoid the position bias of the existing algorithm. The user response is provided as a click on a hotel. From: https://www.kaggle.com/c/expedia-personalized-sort/overview

1. Connect to Fiddler
2. Load a Data Sample
3. Define the Model Specifications and Model Task
4. Create a Model
5. Publish a Pre-production Baseline
6. Publish Production Events

## 0. Imports

In [1]:
%pip install -q fiddler-client

import os
import time as time

import pandas as pd
import fiddler as fdl

print(f"Running Fiddler Python client version {fdl.__version__}")


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Running Fiddler Python client version 3.4.0


## 1. Connect to Fiddler

Before you can add information about your model with Fiddler, you'll need to connect using the Fiddler Python client.


---


**We need a couple pieces of information to get started.**
1. The URL you're using to connect to Fiddler
2. Your authorization token

Your authorization token can be found by navigating to the **Credentials** tab on the **Settings** page of your Fiddler environment.

In [2]:
URL = ''  # Make sure to include the full URL (including https:// e.g. 'https://your_company_name.fiddler.ai').
TOKEN = ''

Constants for this example notebook, change as needed to create your own versions

In [11]:
# Constants for the default example, change as needed to create your own versions
PROJECT_NAME = 'quickstart_examples'
MODEL_NAME = 'search_ranking_example'
DATASET_NAME = 'expedia_dataset'

PATH_TO_SAMPLE_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/expedia_data_sample.csv'
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/expedia_logs.csv'

Next we use these credentials to connect to the Fiddler API.

In [12]:
fdl.init(url=URL, token=TOKEN)

Once you connect, you can create a new project by specifying a unique project name in the fld.Project constructor and calling the `create()` method. If the project already exists, it will load it for use.

In [13]:
try:
    # Create project
    project = fdl.Project(name=PROJECT_NAME).create()
    print(f'New project created with id = {project.id} and name = {project.name}')
except fdl.Conflict:
    # Get project by name
    project = fdl.Project.from_name(name=PROJECT_NAME)
    print(f'Loaded existing project with id = {project.id} and name = {project.name}')

Loaded existing project with id = 70b74177-c712-44b1-b431-2377c1b908ab and name = quickstart_examplesx


## 2. Load a Data Sample

Now we retrieve the Expedia Dataset as a data sample for this model.

In [14]:
sample_data_df = pd.read_csv(PATH_TO_SAMPLE_CSV)
sample_data_df.head()

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,graded_relevance,weekday,week_of_year,hour,minute,time_epoch,early_night,nans_count,score,timestamp
0,28,5,219,219,5739,4,4.5,1,2.64,0.0468,...,no_click,0,48,10,13,1.353925e+09,False,8,0.788386,1712238535749
1,28,5,219,219,11244,2,3.0,1,2.20,0.0105,...,no_click,0,48,10,13,1.353925e+09,False,9,-0.909483,1712238543994
2,28,5,219,219,11726,3,4.0,1,2.64,0.0178,...,no_click,0,48,10,13,1.353925e+09,False,7,-0.432752,1712238552240
3,28,5,219,219,13996,4,4.0,1,2.89,0.0491,...,booked,0,48,10,13,1.353925e+09,False,7,0.897676,1712238560485
4,28,5,219,219,16096,3,4.5,1,1.61,0.0143,...,no_click,0,48,10,13,1.353925e+09,False,9,-0.506159,1712238568731


Fiddler uses this data sample to keep track of important information about your data.
  
This includes **data types**, **data ranges**, and **unique values** for categorical variables.

## 3. Define the Model Specifications and Model Task

To add a Ranking model you must specify the ModelTask as `RANKING` in the model info object.  

Additionally, you must provide the `group_by` argument that corresponds to the query search id. This `group_by` column should be present either in:
- `features` : if it is used to build and run the model
- `metadata_cols` : if not used by the model

Optionally, you can give a `ranking_top_k` number (default is 50). This will be the number of results within each query to take into account while computing the performance metrics in monitoring.  

Unless the prediction column was part of your baseline dataset, you must provide the minimum and maximum values predictions can take in a dictionary format (see below).  

If your target is categorical (string), you need to provide the `target_class_order` argument. If your target is numerical and you don't specify this argument, Fiddler will infer it.   

This will be the list of possible values for the target **ordered**. The first element should be the least relevant target level, the last element should be the most relevant target level.

In [15]:
model_spec = fdl.ModelSpec(
    inputs=list(
        sample_data_df.drop(
            columns=[
                'binary_relevance',
                'score',
                'graded_relevance',
                'position',
                'timestamp',
            ]
        ).columns
    ),
    outputs=['score'],
    targets=['binary_relevance'],
    metadata=['timestamp', 'graded_relevance', 'position'],
)

If you have columns in your ModelSpec which denote **prediction IDs or timestamps**, then Fiddler can use these to power its analytics accordingly.

Let's call them out here and use them when configuring the Model in step 5.

In [16]:
# id_column = '' # Optional: Specify the name of the ID column if you have one
timestamp_column = 'timestamp'

In [17]:
model_task = fdl.ModelTask.RANKING

task_params = fdl.ModelTaskParams(
    group_by='srch_id', top_k=20, target_class_order=[0, 1]
)

## 4. Create a Model

In [18]:
model = fdl.Model.from_data(
    name=MODEL_NAME,
    project_id=project.id,
    source=sample_data_df,
    spec=model_spec,
    task=model_task,
    task_params=task_params,
    event_ts_col=timestamp_column,
)

model.create()
print(f'New model created with id = {model.id} and name = {model.name}')

New model created with id = 9b5266cb-32aa-4610-8e7c-db4da6220e94 and name = search_ranking_example


## 5. Publish a Pre-production Baseline **(Optional)**

We can publish the data sample from earlier to add it as a baseline.

For ranking, we need to ingest all events from a given query or search ID together. To do that, we need to transform the data to a grouped format.  
You can use the `group_by` utility function to perform this transformation.

In [19]:
sample_df_grouped = fdl.utils.helpers.group_by(
    df=sample_data_df, group_by_col='srch_id'
)

baseline_publish_job = model.publish(
    source=sample_df_grouped,
    environment=fdl.EnvType.PRE_PRODUCTION,
    dataset_name=DATASET_NAME,
)
print(
    f'Initiated pre-production environment data upload with Job ID = {baseline_publish_job.id}'
)

# Uncomment the line below to wait for the job to finish, otherwise it will run in the background.
# You can check the status on the Jobs page in the Fiddler UI or use the job ID to query the job status via the API.
# baseline_publish_job.wait()

Initiated pre-production environment data upload with Job ID = 6387b7c8-9a52-4924-a6b8-e3e5ee2431cc


# 6. Publish Events For Monitoring

Fiddler will **monitor this data and compare it to your baseline to generate powerful insights into how your model is behaving**.


---


Each record sent to Fiddler is called **an event**.
  
Let's load some sample events from a CSV file.

In [20]:
production_data_df = pd.read_csv(PATH_TO_EVENTS_CSV)

# Shift the timestamps of the production events to be as recent as today
production_data_df['timestamp'] = production_data_df['timestamp'] + (
    int(time.time() * 1000) - production_data_df['timestamp'].max()
)
production_data_df

,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,binary_relevance,graded_relevance,weekday,week_of_year,hour,minute,early_night,nans_count,score,timestamp
0,81,2013-06-25 15:39:35,5,219,219,5439,2,3.5,1,0.69,...,0,no_click,1,26,15,39,False,7,-2.440277,1731082276837
1,81,2013-06-25 15:39:35,5,219,219,5449,0,4.5,1,0.00,...,0,no_click,1,26,15,39,False,9,-3.078178,1731082314080
2,81,2013-06-25 15:39:35,5,219,219,6997,3,4.5,1,1.10,...,0,no_click,1,26,15,39,False,5,-2.317659,1731082351324
3,81,2013-06-25 15:39:35,5,219,219,10797,2,3.0,0,1.39,...,0,no_click,1,26,15,39,False,1,-2.254402,1731082388568
4,81,2013-06-25 15:39:35,5,219,219,14142,2,3.5,1,2.71,...,0,no_click,1,26,15,39,False,6,-1.519717,1731082425811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16235,74992,2013-06-27 18:39:51,5,77,219,101604,4,4.0,1,1.95,...,0,no_click,3,26,18,39,False,6,-0.080996,1731686927862
16236,74992,2013-06-27 18:39:51,5,77,219,105389,3,4.0,1,2.48,...,0,no_click,3,26,18,39,False,5,-1.012836,1731686965105
16237,74992,2013-06-27 18:39:51,5,77,219,106976,4,4.0,1,2.64,...,0,no_click,3,26,18,39,False,8,-2.436670,1731687002349
16238,74992,2013-06-27 18:39:51,5,77,219,123831,3,0.0,1,2.08,...,0,no_click,3,26,18,39,False,4,-1.577952,1731687039593


Again, let's group the data before sending it to Fiddler.

In [21]:
df_logs_grouped = fdl.utils.helpers.group_by(
    df=production_data_df, group_by_col='srch_id'
)
df_logs_grouped

,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,binary_relevance,graded_relevance,weekday,week_of_year,hour,minute,early_night,nans_count,score,timestamp
0,81,"[2013-06-25 15:39:35, 2013-06-25 15:39:35, 201...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[219, 219, 219, 219, 219, 219, 219, 219, 219, ...","[219, 219, 219, 219, 219, 219, 219, 219, 219, ...","[5439, 5449, 6997, 10797, 14142, 18221, 21495,...","[2, 0, 3, 2, 2, 3, 3, 2, 2, 0, 3, 2, 2, 2, 2, ...","[3.5, 4.5, 4.5, 3.0, 3.5, 4.5, 4.0, 4.0, 2.5, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, ...","[0.69, 0.0, 1.1, 1.39, 2.71, 1.39, 1.39, 3.09,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no_click, no_click, no_click, no_click, no_cl...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 2...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 3...","[False, False, False, False, False, False, Fal...","[7, 9, 5, 1, 6, 5, 5, 1, 3, 13, 7, 9, 3, 10, 6...","[-2.440276941922958, -3.078178407011871, -2.31...","[1731082276837, 1731082314080, 1731082351324, ..."
1,235,"[2013-05-28 09:57:11, 2013-05-28 09:57:11, 201...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[219, 219, 219, 219, 219, 219, 219, 219, 219, ...","[219, 219, 219, 219, 219, 219, 219, 219, 219, ...","[2543, 3469, 5922, 6100, 10708, 14198, 15447, ...","[3, 3, 4, 3, 3, 2, 3, 2, 3, 2, 3, 3, 3, 2, 2, ...","[3.0, 4.5, 4.5, 3.5, 3.5, 3.5, 3.5, 2.0, 4.5, ...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[3.22, 2.3, 0.0, 2.48, 0.69, 2.4, 3.0, 1.39, 2...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[no_click, no_click, no_click, no_click, no_cl...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 5...","[False, False, False, False, False, False, Fal...","[5, 3, 6, 1, 4, 5, 5, 7, 6, 5, 4, 7, 7, 4, 5, ...","[-0.514883076286599, -0.2560329171201618, -0.7...","[1731083617609, 1731083654852, 1731083692096, ..."
2,442,"[2013-06-13 09:09:23, 2013-06-13 09:09:23, 201...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[219, 219, 219, 219, 219, 219, 219, 219, 219, ...","[219, 219, 219, 219, 219, 219, 219, 219, 219, ...","[6658, 7777, 10426, 12383, 19305, 24174, 38516...","[3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 3, ...","[4.0, 3.5, 3.0, 2.5, 3.5, 3.5, 3.0, 4.0, 3.5, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.69, 1.1, 2.08, 1.61, 1.61, 2.48, 2.56, 2.3,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no_click, no_click, no_click, no_click, no_cl...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 2...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[False, False, False, False, False, False, Fal...","[11, 9, 11, 8, 8, 12, 7, 9, 9, 7, 9, 8, 11, 11...","[-1.5810057858952118, -0.8871645585810006, -2....","[1731084883894, 1731084921137, 1731084958381, ..."
3,806,"[2013-05-26 17:13:03, 2013-05-26 17:13:03, 201...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[129, 129, 129, 129, 129, 129, 129, 129, 129, ...","[219, 219, 219, 219, 219, 219, 219, 219, 219, ...","[1272, 4030, 6369, 16412, 17427, 20826, 28067,...","[3, 2, 3, 4, 2, 2, 4, 3, 3, 2, 4, 3, 2, 3, 3, ...","[5.0, 3.5, 3.0, 4.0, 4.0, 3.0, 4.0, 4.5, 4.5, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1.39, 2.4, 1.1, 1.79, 2.4, 1.61, 1.61, 1.39, ...",...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[no_click, no_click, no_click, no_click, no_cl...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 1...","[13, 13,

Now publish the events

In [22]:
production_publish_job = model.publish(df_logs_grouped)

print(
    f'Initiated production environment data upload with Job ID = {production_publish_job.id}'
)

# Uncomment the line below to wait for the job to finish, otherwise it will run in the background.
# You can check the status on the Jobs page in the Fiddler UI or use the job ID to query the job status via the API.
# production_publish_job.wait()

Initiated production environment data upload with Job ID = 120b3bd5-73fc-4802-ab4b-4a20ee797fc8


# Get insights


**You're all done!**
  
You can now head to your Fiddler environment and start getting enhanced observability into your model's performance.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/ranking_model_1.png" />
        </td>
    </tr>
</table>

--------
**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

Join our [community Slack](http://fiddler-community.slack.com/) to ask any questions!

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.